# **Mean-Variance Optimization with risk-free assets**
## *Solution with variance penalty*
--------------------------------------------------
This notebook solves the same problem as notebook MPT-3, except it uses a different approach to get a solution.

Following common practice, it considers the volatility of the market as a measure of risk and favors portfolios having less volatility through a penalty term, favoring choices of asset allocation having the right balance of risk tolerance. The frequency at which changes in asset allocation could be performed can be selected from monthly to yearly, all the way down to only once in the 16-year period that represents the range of historical data considered here and obtained from a public source.

Mathematically, the solution to this  problem consists in maximizing the portfolio return under a variance penalty constraint and a long-only portfolio, i.e., a portfolio in which one can only invest in the assets, not short them. This problem has no analytical solution, but it can easily be solved numerically with modern algorithms. This script only considers 5 different assets: ExxonMobil stock (XOM), S&P 500 (^GSPC), Dow Jones US Completion Total (^DWCPF), MSCI World ex US Market Index (ACWX), Bloomberg Aggregate Bond market (AGG), and risk-free cash (^IRX) represented by 3-month Treasury bills. Only growth is considered, inflation is not. For tracking the US Aggregate Bond market, we use the AGG ETF which has an inception year of 2003.


### Mathematical formulation of the problem
The level of mathematics involved here only requires basic linear algebra, in particular, matrix-vector multiplication, and basic statistics.
Following Markowitz modern portfolio theory, we consider the variance of a market portfolio consisting of assets allocated with weights stored in a vector $w$, and having a covariance matrix typically represented by $\Sigma$ which is calculated between the time series of these assets. The variance on the market-invested portoflio is then expressed as
\\[ \sigma_m^2 = w^T \Sigma w, \\]
where $T$ is the transpose operator changing a column vector into a row vector. The square root of the variance, $\sigma_m$ is the standard deviation that quantifies the volatility. Under this approach, the standard deviation is a measure of risk.

The rate of return on the market part of the portfolio, i.e., excluding the risk-free asset is
\\[ w^T \alpha, \\]
where $\alpha$ is a vector containing the average rates of return for each of the $n$ assets in which the portfolio is invested over the period considered. It is just a weighted sum of average rates, where the weights are a fraction of unity.

Let vector $1_n$ be a vector having 1 for all its elements.
We consider a portfolio which also has a risk-free asset available for investment with a rate of return $r_0$
in which we can invest the remaining fraction $(1 - w^T 1_n)$ not invested in the market.
The objective function that we would like to maximize is the total return of a portfolio that can also invest in a risk-free asset with return $r_0$,
\\[f(w) = w^T \alpha + (1 - w^T 1_n)r_0 -\gamma \sigma^2, \\]
where the variance of the whole portfolio (i.e., considering the risk-free part) is
\\[ \sigma^2 = (w^T 1_m)^2\  w^T\Sigma w,\\]
and under the condition that we only invest, requiring that $w \ge 0$ element-wise, (i.e., no short position), and no borrowing on our cash position,
\\[ w^T 1_n \le 1. \\]
It can be observed that if $w=0$, then the portfolio is totally invested in the risk-free part, the variance is 0, and the return is $r_0$.

The variable $\gamma$ is a measure of aversion to risk, and responds to values between 0 (all in the leading asset for the time period) to about 2,000 (all cash). 100 is conservative, 30 moderate, and 10 is aggressive.

While the inequality constraint on variance is a 4th-order equation, the problem can still be solved using sequential quadratic programming with the inequality constraints and bounds presented here. For this purpose, we  use the scipy library. We run the optimization using historical daily values divided by periods of one to several months depending on the user selection. The time series for the assets are downloaded from Yahoo finance. For each period, optimal weights are calculated and values of the portfolio's annualized rates of return are computed and stored for displaying in graphs at the end of the calculations. We restrict the weights $w$ to be between 0 and 1. We use the 252 days of trading in the year to convert annual rates to trading-day rates. More details on the computation are given in the comments inserted in the Python code below.

Additional bounds can be imposed on the fraction of investment made in the risk-free asset (`maxCash`) and on holding positions for each market asset (`maxAssetFraction`). The data used are the adjusted daily data at closing (adjusted for splits and dividends). Daily data are grouped in time periods represented in multiples of months (`nMonths`). Choosing 12 months gives an optimization that can re-adjust asset allocation once a year, while choosing 192 months only allows for a single set of asset allocation over the 16 year considered. Note that this is not re-balancing as accounts are implicitly assumed to be always in balance with the chosen assets allocation during each period. Choosing a high number such as 96 months (8 years) gives the historical rate of return from a scenario where one chooses a constant allocation ratio over the first 8 years, and another one for the other 8 following years.

The last thing to select is the desired risk aversion $\gamma$ denoted by `gamma`. When selecting long-term periods, such as 96 months, one can easily realize that the tolerance for volatility needs to be relaxed in order to achieve higher rates of returns. Interestingly, for the last 16 years, the optimal long-term asset allocation is not the 60/40 stock/bonds portfolio commonly recommended by financial advisors, but rather something else (cash and stocks). While bonds are part of the solution in downturns years, when `nMonths` $\le$ 96, no choice of volatility yielded a 60/40 stocks/bonds portfolio for a 16-year long-term allocation solution. For 2 blocks of 8 years, the solution has only 2 assets: extended stocks/bonds for the first block, but stock/cash allocation for the second half. This echoes what many analysts have said regarding the fate of the 60/40 portfolio wisdom with recent market performance.

It is hoped that this script will help the investor understand the long-term impact of the choices that were made on asset allocation. But as always, past market performance is no guarantee of future performance. Nevertheless, there are still useful lessons to be learned here.

Enjoy!

&copy;  2024 Martin-D. Lacasse

This notebook comes with no guarantee. Use at your own risk.


#### Additional packages required
This notebook requires the installation of the `yfinance` package. In Anaconda, this can be installed using
    
    conda install -c conda-forge yfinance
or

    pip install yfinance
otherwise.

### The main adjustable parameters

In [ ]:
# Aversion to risk:
gamma = 25

# Period over which to divide and perform analysis in months. Note that the underlying data is daily.
# Pick 1,2,3,6,12,24,32,48,64,96, or 192 as these values all divide the 16 years of historical data.
nMonths = 12

# Maximum fraction to hold in cash.
maxCash = 0.8

# Maximum fraction in any market asset:
maxAssetFraction = 0.85

### Other adjustable parameters 

In [ ]:
# Default is to sync all time periods to today's date.
# Change to True for using Jan 1 as a reference, allowing to compare annual rates with public data.
useJan1 = False

# If you do not want to optimize but rather just use fixed asset allocation:
# Change variable to True and set myWeights accordingly (in decimal).
fixedWeights = False
# Weights are [XOM, S&P_500, DJ_US_Total_Completion, MSCI_WorldexUS, Bonds]
myWeights = [0, .8, 0, 0, 0]

### The code - nothing to adjust below

Just run all cells and look at the graphs below. Adjust the parameters and repeat.

Read the code if you want more details.

-------------------------------------------------------------------------

In [ ]:
import yfinance as yf
import numpy as np
from datetime import datetime, timedelta, date
from scipy.optimize import minimize
import pandas as pd
import matplotlib.pyplot as plt

#### Defining a few useful functions

In [ ]:
def variance(weights, covMatrix):
    '''
    The variance on the market portion of the portfolio.
    This is a vector matrix vector multiplication leading to a scalar.
    '''
    return weights.T @ covMatrix @ weights

In [ ]:
def stdDev(weights, covMatrix):
    '''
    The standard deviation (volatility) of the market portion of the portfolio.
    ''' 
    return np.sqrt(variance(weights, covMatrix))

In [ ]:
def objective(weights, alphas, r0, covMatrix, gamma):
    '''
    This is the (negative) return on a portfolio with a risk-free asset.
    The negative sign is so that this function can be minimized.
    '''
    return -(weights.T@alphas + (1 - np.sum(weights))*r0 - 0.5*gamma*(np.sum(weights)**2)*(weights.T@covMatrix@weights))

In [ ]:
def a2td(ar):
    '''
    Convert an annual rate to a trading-day rate.
    This is used for converting rates to daily trading rates for computation.
    '''
    return (1 + ar)**(1/252) - 1

In [ ]:
def shiftWeekends(date):
    '''
    Avoid requesting data for days on weekend as they cause spurious effects in yfinance.
    Shift day back to the Friday just before weekend.
    '''
    dow = date.strftime('%w')
    if dow == '0': # Sunday
        return date - timedelta(days = 2)
    if dow == '6': # Saturday
        return date - timedelta(days = 1)
    
    return date

#### The loop putting it all together

In [ ]:
tickers = ['XOM', '^GSPC', '^DWCPF', 'ACWX', 'AGG']
alltickers = tickers + ['cash']

now = datetime.now()
if useJan1 == True:
    # Use Jan 1 of this year as reference point.
    refDay = date(now.year, 1, 1)
else:
    # Or start from today.
    refDay = date(now.year, now.month, now.day) 

# We define constraints through dictionaries as required by scipy minimize().
# We limit variance to be below the desired value.
# We ensure market total weights are below 1, the rest being in cash, but smaller than maxCash. 
# Inequality expressions are meant to be >= 0.
constraints = [
               {'type': 'ineq', 'fun': lambda weights: 1 - np.sum(weights) },
               {'type': 'ineq', 'fun': lambda weights: maxCash - 1 + np.sum(weights)  }
              ]

if fixedWeights == False:
    # We only consider long positions (no short, i.e., >= 0) with a maximum asset weight (<= maxAssetFraction).
    bounds = [(0., maxAssetFraction) for _ in range(len(tickers))]
else:    
    # You requested fixed bounds to test the performance of fixed-ratio portfolios.
    # To ensure feasibility, we override the desired volatility to a high number (e.g.. > 100%).
    desiredVolatility = 100
    # Use the values provided by the user as low and high bounds.
    bounds = []
    for i in range(length(myWeights)):
        bounds.append((myWeights[i], myWeights[i]))

# We create four dataframes for storing results:
computedWeights = pd.DataFrame(columns=alltickers)
assetsAnnualReturns = pd.DataFrame(columns=alltickers)
portfolioPeriodReturns = pd.DataFrame(columns=['return'])
portfolioAnnualReturns = pd.DataFrame(columns=['return'])

# Note that Yahoo finance does not provide daily data before 2008 for some indices here.
totalMonths = 16*12
if totalMonths%nMonths != 0:
    raise Exception("Sorry, nMonths must be a divider of", totalMonths)
    
# A list for storing timestamps of the beginning of each period.
years = []

startDate = refDay - timedelta(days = int(round((totalMonths)*365.25/12)))
for monthsAgo in range(totalMonths-nMonths, -1, -nMonths):
    # Periods must overlap by one day. See below where startDate gets reset to endDate - 1.
    endDate = refDay - timedelta(days = int(round((monthsAgo)*365.25/12)))
    # Avoid weekends.
    startDate = shiftWeekends(startDate)
    endDate = shiftWeekends(endDate)
    print('From', startDate, 'to', endDate)
    years.append(startDate)

    # Download the data.
    adjCloseDf = pd.DataFrame()
    for ticker in tickers:
        data = yf.download(ticker, start=startDate, end=endDate)
        adjCloseDf[ticker] = data['Adj Close']

    adjCloseDf = adjCloseDf.dropna()
    # print(adjCloseDf)
       
    # Daily market return is (closing - previous-day closing)/previous-day closing.
    dailyReturns = (adjCloseDf/adjCloseDf.shift(1)) - 1
    # Drop NA first row as first day of the set has no previous day.
    dailyReturns = dailyReturns.dropna()
    # print(dailyReturns)
    # Compute covariance matrix between returns of all requested tickers for the period considered.
    covMatrix = dailyReturns.cov()
    # print(covMatrix)
    # Diagonal of covariance matrix is the variance on that asset.
    print("Volatility: ", np.sqrt(np.diag(covMatrix)))
    
    # For each ticker, compute the returns for the selected multi-month period using first and last data points.
    # Then convert to mean trading-day daily returns.
    meanAnnualReturns = []
    meanDailyReturns = []
    meanPeriodReturns = []
    # print(adjCloseDf.index[0], '->', adjCloseDf.index[-1])
    for ticker in tickers:
        # print(ticker, adjCloseDf[ticker].iloc[0], '->', adjCloseDf[ticker].iloc[-1])
        pr = (adjCloseDf[ticker].iloc[-1]/adjCloseDf[ticker].iloc[0]) -1
        # Convert period growth to annual growth.
        ar = (1 + pr)**(12/nMonths) - 1
        # Then convert annualized rate to mean trading-day rate.
        mdr = a2td(ar)
        meanDailyReturns.append(mdr)
        meanAnnualReturns.append(ar)
        meanPeriodReturns.append(pr)

    # Download short-term percent rates for treasury bills.
    # We wll use this rate as the annual rate on our risk-free asset.
    data = yf.download('^IRX', start=startDate, end=endDate)
    # Reset startDate for next period.
    startDate = endDate - timedelta(days = 1)

    # Take the mean over period and convert from percent to decimal.
    ar0 = data['Adj Close'].mean()/100
    print('Annualized risk-free r0: %.2f%%'%(100*ar0))
    # Return rate over period.
    pr0 = (1 + ar0)**(nMonths/12) - 1
    # Convert from annualized return to return per trading day.
    dr0 = a2td(ar0)
        
    # Convert lists to arrays: Some for humans to understand.
    palphas = np.array(meanPeriodReturns)
    print('Assets returns over period (%):', (100*palphas))
    alphas = np.array(meanAnnualReturns)
    print('Annualized returns over period (%):', (100*alphas))
    # The other one for the optimization steps where everyting is expressed in trading days. 
    dalphas = np.array(meanDailyReturns)
    
    # Start with a solution that is mostly in cash.
    initialWeights = np.array([0.1]*len(tickers))
    # Solve
    solution = minimize(objective, initialWeights, args=(dalphas, dr0, covMatrix, gamma), constraints=constraints,
                        bounds=bounds, tol=1e-14, options={'maxiter':1000})
    if solution.success != True:
        print('WARNING: Optimization failed: ', solution.message, solution.success)  
        
    optWeights = solution.x
    marketVolatility = np.sqrt(252)*stdDev(optWeights, covMatrix)
    actualVolatility = (np.sum(optWeights))*marketVolatility
    optPeriodReturn = -objective(optWeights, palphas, pr0, covMatrix, gamma)
    print('Expected period return of %.2f%% with volatility %.1f%%'%(100*optPeriodReturn, 100*actualVolatility))
    print('Market volatility %.1f%%'%(100*marketVolatility))
    
    # Add cash weight to asset allocation.
    computedWeights.loc[len(computedWeights)] = np.append(optWeights, (1 - sum(optWeights)))
    # Add cash annual return to our list.
    assetsAnnualReturns.loc[len(assetsAnnualReturns)] = np.append(alphas, ar0)
    portfolioPeriodReturns.loc[len(portfolioPeriodReturns)] = [optPeriodReturn]
    annReturn = (1 + optPeriodReturn)**(12/nMonths) - 1
    portfolioAnnualReturns.loc[len(portfolioAnnualReturns)] = [annReturn]
    
# Generate a new index from timestamps marking beginning of period.
newIndex = {}
for i in range(len(years)):
    newIndex[i] = years[i].strftime('%Y.%m.%d')
# and use to replace default index.  
computedWeights.rename(index=newIndex, inplace=True)
assetsAnnualReturns.rename(index=newIndex, inplace=True)
portfolioPeriodReturns.rename(index=newIndex, inplace=True)
portfolioAnnualReturns.rename(index=newIndex, inplace=True)

print('Optimal weights:')
pd.set_option('display.width', 100)
print(computedWeights)

# Take geometric mean of returns over all periods.
prates = portfolioPeriodReturns.to_numpy() + 1
cumGain = prates.prod()
print('Cumulative gain over epoch: %.1f%%'%(100*(cumGain -1)))
meanPeriodReturn = (np.exp(np.log(cumGain)/len(prates)) -1)
# Then annualize it.
meanAnnualReturn = (1 + meanPeriodReturn)**(12/nMonths) - 1

print('Mean period return: %.2f%%'%(100*meanPeriodReturn))
print('Mean annual return: %.2f%%'%(100*meanAnnualReturn))
# print('Period', portfolioPeriodReturns)
# print('Annual', portfolioAnnualReturns)

### Plot of efficient portfolios

In [ ]:
# For the next line to work, LaTeX must be installed on the host computer. 
# plt.rcParams['text.usetex'] = True
plt.rcParams["figure.figsize"] = (14, 6)

fig, ax = plt.subplots()
assetsAnnualReturns.plot(ax=ax, kind='bar', legend=True)
ax.set(xlabel='period starting date', ylabel='return (decimal)',
    title='Annualized rates of return by %d-month periods'%nMonths)
ax.set_xticklabels(ax.get_xticklabels(), rotation=60, ha='right')
ax.grid()
plt.show()

fig, ax = plt.subplots()
computedWeights.plot(ax=ax, kind='bar', legend=True)
ax.set(xlabel='period starting date', ylabel='weights (decimal)',
       title='Optimal asset allocation with risk-free asset and gamma = '+str(gamma))
ax.set_xticklabels(ax.get_xticklabels(), rotation=60, ha='right')
ax.grid()
plt.show()

fig, ax = plt.subplots()
portfolioAnnualReturns.plot(ax=ax, kind='bar', legend=True)
ax.set(xlabel='period starting date', ylabel='return (decimal)',
    title='Return of optimal asset allocation with risk-free asset and gamma = %.0f, <mean> %.2f%%'%(gamma, 100*meanAnnualReturn))
ax.set_xticklabels(ax.get_xticklabels(), rotation=60, ha='right')
ax.grid()
plt.show()
